In [1]:
import re
import os

import joblib
import numpy as np
import openai
import pandas as pd
from bertopic import BERTopic
from bertopic.backend import OpenAIBackend
from bertopic.representation import (KeyBERTInspired, OpenAI,
                                     ZeroShotClassification)
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

In [2]:
df = pd.read_csv('input/flexitarianism.csv')
df

,id,motivo_de_se_considerar_flex,fonteinfo_flex,motivacao,dificuldade,genero,cor_raca,escolaridade
0,1,nao me considero,amigos,aversao,condenacao moral,mulher_cis,branca,superior
1,2,reducao do impacto_ambiental,escola_universidade,impacto_ambiental,habito cultural adquirido,homem_cis,branca,doutorado
2,3,porque a carne bovina de certa forma me faz ma...,amigos,saude,preco e variedade de outras proteinas vegetais...,homem_cis,branca,mestrado
3,4,financeiro,amigos,religiao,consumo desde crianca,pnr,parda,superior
4,5,pois limitei o consumo para nao desenvolver ac...,midias,saude,nenhuma,mulher_cis,branca,superior
...,...,...,...,...,...,...,...,...
1024,1025,necessidade financeira,midias,saude,nenhuma,mulher_cis,preta,medio
1025,1026,o amor pelos animais,escola_universidade,etica_animal,habito alimentae,mulher_cis,branca,superior
1026,1027,porque eu deliberadamente tenho reduzido meu c...,midias,saude,as pessoas fora da minha casa nao estarem envo...,homem_cis,branca,medio
1027,1028,"porque cuido de mim, do meu corpo e do meu pla...",escola_universidade,impacto_ambiental,nenhuma,mulher_cis,branca,superior


In [3]:
df_reason = df.copy()


In [4]:
# Rename the column 'motivo_de_se_considerar_flex' to 'input_text' in the df_reason DataFrame
# This standardizes the column name for easier processing
df_reason.rename(columns={'motivo_de_se_considerar_flex': 'input_text'}, inplace=True)

# Calculate the length of the text in the 'input_text' column for each row in df_reason
# Store the lengths in a new column called 'len_text'
df_reason['len_text'] = df_reason.input_text.str.len()

# Calculate the 25th percentile (first quartile) of the text lengths in df_reason
# This value will be used to filter out shorter texts
reason_quartile = df_reason.len_text.quantile(0.25)

# Filter df_reason to include only rows where the text length is greater than or equal to the first quartile
# This removes the shortest 25% of texts
df_reason = df_reason.query('len_text >= @reason_quartile')

# Drop rows with missing values in the 'input_text' column from df_reason
# This ensures that all remaining rows have valid text data
df_reason.dropna(inplace=True, subset=['input_text'])


In [5]:
!python -m spacy download pt_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 41.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


## Cleaning our data and saving the transformed data

In [6]:
import re
from sklearn.pipeline import Pipeline # Pipeline applies a list of transforms. You can also add an estimator at the end, so it will be completely encapsulated.
from sklearn.preprocessing import FunctionTransformer # FunctionTransformer allows to apply an arbitrary function to the data, so we can use it in the pipeline
import unicodedata
import spacy
from typing import List

def remove_excessive_spaces(text: str) -> str:
    """
    This function removes excessive spaces from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with excessive spaces removed.
    """
    return re.sub(r'\s+', ' ', text).strip() 

def remove_repeated_non_word_characters(text: str) -> str:
    """
    This function removes repeated non-word characters from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with repeated non-word characters removed.
    """
    return re.sub(r'(\W)\1+', r'\1', text).strip()

def remove_first_line_from_text(text: str) -> str:
    """
    This function removes the first line from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with the first line removed.
    """
    return re.sub(r'^.*\n', '', text).strip()

def remove_last_line_from_text(text: str) -> str:
    """
    This function removes the last line from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with the last line removed.
    """
    return re.sub(r'\n.*$', '', text).strip()

def fix_isolated_commas_in_text(text: str) -> str:
    """
    This function fixes isolated commas in the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with isolated commas fixed.
    """
    text = re.sub(r' ([.,:;!?])', r'\1', text)
    return text.strip()

def keep_words_longer_than(text: str, min_length: int = 2) -> str:
    """
    This function keeps only the words in the text that are longer than a given length.

    Args:
        text (str): The input text.
        min_length (int, optional): The minimum length of the words to keep. Defaults to 2.

    Returns:
        str: The text with only the words longer than the given length.
    """
    return ' '.join([word for word in text.split() if len(word) > min_length])

def keep_only_alphabet_characters(text: str) -> str:
    """
    This function keeps only the alphabet characters in the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with only the alphabet characters.
    """
    return re.sub(r'[^a-zA-Z]', ' ', text).strip()

def remove_accents_from_text(text: str) -> str:
    """
    This function removes accents from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with accents removed.
    """
    return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')

def lemmatize_text_with_spacy(text: str) -> str:
    """
    This function lemmatizes the text using the Spacy library.

    Args:
        text (str): The input text.

    Returns:
        str: The lemmatized text.
    """
    doc = nlp_spacy(text)
    return ' '.join([token.lemma_ for token in doc])


pipeline_clean_text = Pipeline([
    ('remove_first_line_from_text', FunctionTransformer(remove_first_line_from_text)),
    ('remove_last_line_from_text', FunctionTransformer(remove_last_line_from_text)),
    ('remove_excessive_spaces', FunctionTransformer(remove_excessive_spaces)),
    ('remove_repeated_non_word_characters', FunctionTransformer(remove_repeated_non_word_characters)),
    ('fix_isolated_commas_in_text', FunctionTransformer(fix_isolated_commas_in_text)),
])


nlp_spacy = spacy.load('pt_core_news_sm')



In [7]:
# We can apply the pipeline to the data
df_reason['input_text_clean'] = df_reason['input_text'].apply(pipeline_clean_text.transform)

df_reason['input_text_clean_simplified'] = df_reason['input_text_clean'].apply(lemmatize_text_with_spacy)
df_reason['input_text_clean_simplified'] = df_reason['input_text_clean_simplified'].apply(remove_accents_from_text)
df_reason['input_text_clean_simplified'] = df_reason['input_text_clean_simplified'].apply(keep_words_longer_than)
df_reason['input_text_clean_simplified'] = df_reason['input_text_clean_simplified'].str.lower()
df_reason['input_text_clean_simplified'] = df_reason['input_text_clean_simplified'].apply(keep_only_alphabet_characters)
df_reason['input_text_clean_simplified'] = df_reason['input_text_clean_simplified'].apply(remove_excessive_spaces)


In [8]:
# Print the shape of the df_reason DataFrame before dropping rows with missing values
# This shows the number of rows and columns before the cleaning process
print(df_reason.shape)

# Drop rows from df_reason where the 'input_text_clean' column has missing values
# This ensures that all remaining rows have valid cleaned text data
df_reason.dropna(inplace=True, subset=['input_text_clean'])

# Print the shape of the df_reason DataFrame after dropping rows with missing values
# This shows the number of rows and columns after the cleaning process, allowing comparison with the previous shape
print(df_reason.shape)

(778, 11)
(778, 11)


In [9]:
# Import the spaCy library for advanced natural language processing tasks
import spacy

# Import the stopwords module from the nltk.corpus package
# This module provides a list of common stop words for various languages
from nltk.corpus import stopwords

# Get the list of Portuguese stop words from the NLTK library
# These are common words that are usually filtered out in text processing
stopwords_nltk = stopwords.words('portuguese')

# Load the small Portuguese language model from spaCy
# This model includes pre-trained word vectors, part-of-speech tags, named entity recognition, and more
nlp = spacy.load('pt_core_news_sm')

# Access the set of default stop words for the Portuguese language from the loaded spaCy model
stopwords_spacy = nlp.Defaults.stop_words

# Combine the stop words from both NLTK and spaCy into a single set
# Using a set ensures that each stop word appears only once, even if it is present in both lists
both_stopwords = set(stopwords_nltk) | set(stopwords_spacy)

# Calculate and display the total number of unique stop words in the combined set
len(both_stopwords)

500

In [10]:
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [11]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv

# Load environment variables from a .env file. This is necessary to access the OPENAI_API_KEY environment variable.
load_dotenv()

openai_client = ChatOpenAI(model='gpt-4o-mini')
max_tokens = 8191  # the maximum for text-embedding-3-large is 8191


In [12]:
# Check if any text in the 'input_text_clean' column of df_reason is longer than max_tokens
# This is done to ensure that the texts do not exceed the token limit, which might be required for certain NLP models or APIs
# openai_client.get_num_tokens(text) returns the number of tokens in the given text
long_texts_reason = [
    text for text in df_reason['input_text_clean'] 
    if openai_client.get_num_tokens(text) > max_tokens
]


# Print the number of texts longer than max_tokens for both reasons and challenges
# This provides a quick summary of how many texts exceed the token limit in each dataset
print(f'Number of texts longer than {max_tokens} tokens: {len(long_texts_reason)} (reasons)')

Number of texts longer than 8191 tokens: 0 (reasons)


In [13]:
# According to OpenAI pricing website, the cost of the embedding API is $0.13 per million tokens
# https://openai.com/pricing/

from typing import List

def calculate_total_cost(text_list: List[str], cost_per_million_tokens: float) -> float:
    """
    This function calculates the total cost of processing a list of texts, 
    based on a cost per million tokens.

    Args:
        text_list (List[str]): A list of texts to be processed.
        cost_per_million_tokens (float): The cost of processing a million tokens.

    Returns:
        float: The total cost of processing the texts.
    """
    # Initialize the total number of tokens
    total_token_count = 0

    # Iterate over each text in the list
    for text in text_list:
        # Use the OpenAI client to get the number of tokens in the text
        # and add it to the total token count
        total_token_count += openai_client.get_num_tokens(text)

    # Calculate the total cost by multiplying the total token count by the cost per million tokens
    # and dividing by 1,000,000 (since the cost is per million tokens)
    total_cost = total_token_count * cost_per_million_tokens / 1_000_000

    return total_cost

# Concatenate the 'input_text_clean' columns from df_reason and df_challenge into a single list
# This combines the cleaned text data from both DataFrames for cost calculation
all_texts_concat = df_reason['input_text_clean'].values.tolist()

# Calculate and print the total cost of using the embedding API for the concatenated texts
# The cost is calculated using the calculate_total_cost function with a rate of $0.13 per million tokens
print(f'Cost of embedding API for {len(all_texts_concat)} texts: ${calculate_total_cost(all_texts_concat, 0.13):.4f} USD')

Cost of embedding API for 778 texts: $0.0027 USD


In [14]:
df_reason

,id,input_text,fonteinfo_flex,motivacao,dificuldade,genero,cor_raca,escolaridade,len_text,input_text_clean,input_text_clean_simplified
2,3,porque a carne bovina de certa forma me faz ma...,amigos,saude,preco e variedade de outras proteinas vegetais...,homem_cis,branca,mestrado,123,porque a carne bovina de certa forma me faz ma...,porque carne bovino certo forma fazer mal deix...
4,5,pois limitei o consumo para nao desenvolver ac...,midias,saude,nenhuma,mulher_cis,branca,superior,138,pois limitei o consumo para nao desenvolver ac...,pois limitar consumo para nao desenvolver acid...
6,7,so como carne quando nao ha alternativa nao ti...,amigos,impacto_ambiental,gostar do sabor,mulher_cis,branca,mestrado,297,so como carne quando nao ha alternativa nao ti...,como carne quando nao alternativo nao ter pens...
7,8,ja uns anos desde a adolescencia (nos anos 200...,midias,aversao,"na adolescencia e de novo na vida adulta, eu o...",mulher_cis,branca,mestrado,426,ja uns anos desde a adolescencia (nos anos 200...,ano desde adolescencia ano optar por reduzir c...
8,9,porque ao longo do tempo entendi que a reducao...,amigos,impacto_ambiental,nenhuma,mulher_cis,preta,superior,153,porque ao longo do tempo entendi que a reducao...,porque longo tempo entendi que reducao carne n...
...,...,...,...,...,...,...,...,...,...,...,...
1022,1023,consumo carne apenas na refeicao do almoco,escola_universidade,impacto_ambiental,substitulo que se iguale ao sabor,mulher_cis,branca,medio,42,consumo carne apenas na refeicao do almoco,consumo carne apenas refeicao almoco
1023,1024,porque nao consumo carne em todas as refeicões,outros,saude,gostar do sabor e nao poder retirar a carne po...,mulher_cis,branca,doutorado,46,porque nao consumo carne em todas as refeicões,porque nao consumo carne todo refeicao
1026,1027,porque eu deliberadamente tenho reduzido meu c...,midias,saude,as pessoas fora da minha casa nao estarem envo...,homem_cis,branca,medio,61,porque eu deliberadamente tenho reduzido meu c...,porque deliberadamente ter reduzir meu consumo...
1027,1028,"porque cuido de mim, do meu corpo e do meu pla...",escola_universidade,impacto_ambiental,nenhuma,mulher_cis,branca,superior,50,"porque cuido de mim, do meu corpo e do meu pla...",porque cuer meu corpo meu planeta


In [15]:
# Initialize the OpenAI embeddings model with the specified parameters
# - model: The name of the model to use for generating embeddings ('text-embedding-3-large')
# - dimensions: The dimensionality of the embeddings (3072)
# This model will convert text into high-dimensional vectors that capture semantic meaning
openai_embeddings = OpenAIEmbeddings(model='text-embedding-3-large', dimensions=3072)

# Generate embeddings for the 'input_text_clean' column of df_reason
# embed_documents takes a list of texts and returns their embeddings
# These embeddings can be used for various downstream tasks such as clustering, classification, or similarity search
embeddings_reason = openai_embeddings.embed_documents(df_reason['input_text_clean'].values.tolist())


In [16]:
# Convert the list of embeddings for df_reason into a pandas Series and then to a DataFrame
# This creates a DataFrame where each row corresponds to an embedding
embeddings_df_reason = pd.Series(embeddings_reason).to_frame()

# Rename the column of the embeddings DataFrame for df_reason to 'openai_embedding'
# This makes it clear that the column contains embeddings generated by the OpenAI model
embeddings_df_reason.columns = ['openai_embedding']

# Display the embeddings DataFrame for df_reason
# This allows you to inspect the DataFrame and verify that the embeddings have been correctly converted and stored
embeddings_df_reason

,openai_embedding
0,"[0.006274246610701084, 0.014357184059917927, -..."
1,"[0.009271223098039627, 0.028493599966168404, -..."
2,"[0.006935932207852602, 0.025884093716740608, -..."
3,"[0.01741289719939232, 0.008717957884073257, -0..."
4,"[0.02900986559689045, 0.009972958825528622, -0..."
...,...
773,"[-0.008207517676055431, 0.027654049918055534, ..."
774,"[0.02431667409837246, 0.027790484949946404, -0..."
775,"[0.028141194954514503, -0.004925278015434742, ..."
776,"[0.01270632166415453, 0.014720458537340164, -0..."


In [17]:
# Calculate the length of the embedding vector for the first row in the embeddings_df_reason DataFrame
# - iloc[0] selects the first row of the DataFrame
# - ['openai_embedding'] accesses the 'openai_embedding' column, which contains the embedding vector
# - len() calculates the length of the embedding vector
# This is useful to verify the dimensionality of the embeddings generated by the OpenAI model
embedding_length = len(embeddings_df_reason.iloc[0]['openai_embedding'])

# Display the length of the embedding vector
# This confirms that the embeddings have the expected dimensionality (e.g., 3072 dimensions)
embedding_length

3072

In [18]:
df_reason.dropna(subset=['input_text'], inplace=True)

In [19]:
texts = df_reason.input_text.values.tolist()
texts[:5]

['porque a carne bovina de certa forma me faz mal deixa o intestino "pesado" e prefiro nao consumir caso tenham outras opcões',
 'pois limitei o consumo para nao desenvolver acido úrico e por ver muitas reportagens sobre os maleficios do alto consumo de carne vermelha',
 'so como carne quando nao ha alternativa nao tinha pensado ou conhecia essa terminologia sempre me denominei carnivora gosto de carne mas minha filha virou vegetariana entao eu acabei me adaptando e so como carne quando da vontade tipo em marco ate agora comi umas 5 vezes e hj ja e dia 24 de marco',
 'ja uns anos desde a adolescencia (nos anos 2000) eu optei por reduzir o consumo de carne vermelha por nao gostar do sabor e de como eu fico depois que consumo o que nao ocorre tanto quando eu como frango porco (um pouco por causa da gordura) e peixe exceto da forma como as comidas sao temperadas ou cozidas so que tambem nao e algo que me faca falta e mesmo assim de vez em quando fico umas semanas sem consumir carne nenhum

In [24]:
# Initialize embeddings model
embedding_model_langchain = OpenAIEmbeddings(model="text-embedding-3-large")


In [25]:
embeddings = np.array(embeddings_reason)

In [30]:

prompt = """
Eu tenho um tópico que contém os seguintes documentos:
[DOCUMENTS]
O tópico é descrito pelas seguintes palavras-chave: [KEYWORDS]

Com base nas informações acima, extraia um rótulo de tópico curto no seguinte formato:
topic: <topic label>

O rótulo do tópico deve ser uma descrição concisa do conteúdo dos documentos e palavras-chave fornecidos, não fazendo referência a informações específicas de anos, datas, números ou nomes de pessoas. O rótulo do tópico deve ter até 6 palavras.
"""

# Initialize OpenAI client
openai_client = openai.Client()


# Initialize embedding model for BERTopic
embedding_model_bertopic = OpenAIBackend(client=openai_client, embedding_model="text-embedding-3-large") # Same as embedding_model_langchain, that was used to pre-compute the embeddings of our history

# Initialize BERTopic model with specified parameters
representation_model = OpenAI(openai_client, model="gpt-4o", chat=True, prompt=prompt) # This class comes from the bertopic.representation module

In [31]:
topic_model = BERTopic(
    embedding_model=embedding_model_bertopic,
    min_topic_size=15,
    zeroshot_topic_list=["exclui alguns animais", "não come carne vermelha", "diminui a frequência de consumo", "não considera carne essencial"],
    zeroshot_min_similarity=0.8,
    nr_topics=8,
    representation_model=representation_model # this will make len(topics) requests to the OpenAI API
) # KeyBERTInspired

topics, _ = topic_model.fit_transform(texts, embeddings=embeddings)

In [32]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,242,-1_Redução do Consumo de Carne,[Redução do Consumo de Carne],[eu sou consciente dos maleficios do consumo d...
1,0,39,0_Redução do Consumo de Carne,[Redução do Consumo de Carne],[eu nao me dou esse nome porque essa represent...
2,1,56,1_Dieta Pescetariana e Redução de Carnes,[Dieta Pescetariana e Redução de Carnes],[pois deduzo o consumo de carne ja nao como va...
3,2,49,2_Transição e Desafios do Vegetarianismo/Vegan...,[Transição e Desafios do Vegetarianismo/Vegani...,[porque tento me alimentar de forma intuitiva ...
4,3,300,3_Redução do Consumo de Carne,[Redução do Consumo de Carne],[porque tenho consciencia de como a criacao de...
5,4,22,4_Experiências negativas com carne vermelha,[Experiências negativas com carne vermelha],[o gosto de sangue me da enjoo na carne vermel...
6,5,53,5_Redução do Consumo de Carne Semanalmente,[Redução do Consumo de Carne Semanalmente],[porque tento diminuir o consumo de carne e na...
7,6,17,6_Preferências Alimentares e Consumo de Carne,[Preferências Alimentares e Consumo de Carne],[porque como pouca quantidade de proteina anim...


In [33]:
topic_model.visualize_documents(texts, embeddings=embeddings)

In [34]:
gender_topics_reason = df_reason['genero']
education_topics_reason = df_reason['escolaridade']
motivation_topics_reason = df_reason['motivacao']
df_reason

,id,input_text,fonteinfo_flex,motivacao,dificuldade,genero,cor_raca,escolaridade,len_text,input_text_clean,input_text_clean_simplified
2,3,porque a carne bovina de certa forma me faz ma...,amigos,saude,preco e variedade de outras proteinas vegetais...,homem_cis,branca,mestrado,123,porque a carne bovina de certa forma me faz ma...,porque carne bovino certo forma fazer mal deix...
4,5,pois limitei o consumo para nao desenvolver ac...,midias,saude,nenhuma,mulher_cis,branca,superior,138,pois limitei o consumo para nao desenvolver ac...,pois limitar consumo para nao desenvolver acid...
6,7,so como carne quando nao ha alternativa nao ti...,amigos,impacto_ambiental,gostar do sabor,mulher_cis,branca,mestrado,297,so como carne quando nao ha alternativa nao ti...,como carne quando nao alternativo nao ter pens...
7,8,ja uns anos desde a adolescencia (nos anos 200...,midias,aversao,"na adolescencia e de novo na vida adulta, eu o...",mulher_cis,branca,mestrado,426,ja uns anos desde a adolescencia (nos anos 200...,ano desde adolescencia ano optar por reduzir c...
8,9,porque ao longo do tempo entendi que a reducao...,amigos,impacto_ambiental,nenhuma,mulher_cis,preta,superior,153,porque ao longo do tempo entendi que a reducao...,porque longo tempo entendi que reducao carne n...
...,...,...,...,...,...,...,...,...,...,...,...
1022,1023,consumo carne apenas na refeicao do almoco,escola_universidade,impacto_ambiental,substitulo que se iguale ao sabor,mulher_cis,branca,medio,42,consumo carne apenas na refeicao do almoco,consumo carne apenas refeicao almoco
1023,1024,porque nao consumo carne em todas as refeicões,outros,saude,gostar do sabor e nao poder retirar a carne po...,mulher_cis,branca,doutorado,46,porque nao consumo carne em todas as refeicões,porque nao consumo carne todo refeicao
1026,1027,porque eu deliberadamente tenho reduzido meu c...,midias,saude,as pessoas fora da minha casa nao estarem envo...,homem_cis,branca,medio,61,porque eu deliberadamente tenho reduzido meu c...,porque deliberadamente ter reduzir meu consumo...
1027,1028,"porque cuido de mim, do meu corpo e do meu pla...",escola_universidade,impacto_ambiental,nenhuma,mulher_cis,branca,superior,50,"porque cuido de mim, do meu corpo e do meu pla...",porque cuer meu corpo meu planeta


In [35]:
topics_per_gender_reason = topic_model.topics_per_class(texts, classes=gender_topics_reason)
topics_per_gender_reason

,Topic,Words,Frequency,Class
0,-1,Consumo de Carne e Alternativas Alimentares,6,pnr
1,0,Reflexões sobre Aprendizado Acadêmico,1,pnr
2,2,Desafios financeiros para vegetarianismo,1,pnr
3,3,Impacto Ambiental da Pecuária e Consumo,7,pnr
4,4,Dificuldades na Digestão de Carne,2,pnr
5,5,Consumo Reduzido de Carne Diária,2,pnr
6,-1,Desafios do Veganismo e Nutrição,3,nao_binario
7,0,Redução no Consumo de Carne,2,nao_binario
8,3,Redução do Consumo de Carne,2,nao_binario
9,-1,Consumo Sustentável e Práticas Alimentares,50,homem_cis


In [36]:
topics_per_education_reason = topic_model.topics_per_class(texts, classes=education_topics_reason)
topics_per_education_reason

,Topic,Words,Frequency,Class
0,-1,Redução do Consumo de Carnes,31,doutorado
1,0,Descoberta de Dietas Alimentares Alternativas,6,doutorado
2,1,Dieta sem carne vermelha,9,doutorado
3,2,Desafios de Dieta Vegetariana/Vegana,3,doutorado
4,3,Redução do Consumo de Carne,53,doutorado
5,4,Preferências Alimentares e Carne,1,doutorado
6,5,Redução do Consumo de Carne,11,doutorado
7,6,Flexitarianismo e Consumo de Carne,1,doutorado
8,-1,Redução do Consumo de Carne,57,mestrado
9,0,Redução de Consumo de Carne,10,mestrado


In [37]:
topics_per_motivation_reason = topic_model.topics_per_class(texts, classes=motivation_topics_reason)
topics_per_motivation_reason

,Topic,Words,Frequency,Class
0,-1,Redução e Preferências Alimentares sobre Carne,19,aversao
1,1,Consumo Seletivo de Proteínas Animais,5,aversao
2,2,Transição para Dieta Vegana,1,aversao
3,3,Redução do consumo de carne,6,aversao
4,4,Problemas digestivos com carne vermelha,6,aversao
5,5,Redução do Consumo de Carne,1,aversao
6,6,Preferências Alimentares e Consumo de Carne,2,aversao
7,-1,Consumo e ética da carne,54,etica_animal
8,0,Mudança de Dieta para Vegetarianismo,5,etica_animal
9,1,Dietas alimentares sem carne vermelha,18,etica_animal


In [38]:
fig_topics_per_gender_reason = topic_model.visualize_topics_per_class(topics_per_gender_reason, normalize_frequency=True, top_n_topics=50, width=1600, height=900)
fig_topics_per_gender_reason.show()

In [39]:
fig_topics_per_education_reason = topic_model.visualize_topics_per_class(topics_per_education_reason, normalize_frequency=True, top_n_topics=50, width=1600, height=900)
fig_topics_per_education_reason.show()

In [40]:
fig_topics_per_motivation_reason = topic_model.visualize_topics_per_class(topics_per_motivation_reason, normalize_frequency=True, top_n_topics=50, width=1600, height=900)
fig_topics_per_motivation_reason.show()